In [ ]:
#default_exp trainer 

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
#export
import torch, wandb, gc, numpy as np, pandas as pd,os
from wandb.data_types import Histogram
from tqdm.auto import tqdm
from travis_attack.utils import (timecode, show_gpu, merge_dicts, unpack_nested_lists_in_df, 
                                 display_all, append_df_to_csv, start_wandb_run)
from travis_attack.tests import check_no_nans_or_infs
from travis_attack.models import save_pp_model, resume_pp_model, get_vm_probs, get_start_end_special_token_ids, get_nli_probs


In [ ]:
#export
import torch, numpy as np, pandas as pd, gc,sys, warnings
from torch.utils.data import DataLoader, RandomSampler
from datasets import load_dataset, load_metric, load_from_disk, DatasetDict, Dataset
from transformers import (AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, 
                          AutoTokenizer, AdamW, SchedulerType, get_scheduler)
from torch.distributions import Categorical
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import pytorch_cos_sim
from collections import defaultdict
from accelerate import Accelerator, notebook_launcher
from cachetools import cached, LRUCache
from types import MethodType
from timeit import default_timer as timer
from tqdm.auto import tqdm
import itertools
import copy 
import wandb
from undecorated import undecorated
from fastcore.basics import store_attr

import logging 
logger = logging.getLogger("travis_attack.trainer")

from IPython.core.debugger import set_trace


In [ ]:
#hide
from travis_attack.utils import set_seed, set_session_options, setup_logging
from travis_attack.config import Config
from travis_attack.models import prepare_models, get_optimizer
from travis_attack.data import ProcessedDataset
from accelerate import Accelerator, notebook_launcher

from snoop import pp
import snoop

# Trainer 

This class is used to fine-tune the model 

In [ ]:
#export 
class Trainer: 
    def __init__(self, cfg, vm_tokenizer, vm_model, pp_tokenizer, pp_model, ref_pp_model,
                 sts_model, nli_tokenizer, nli_model, optimizer, 
                 ds, initial_eval=True, log_code=True, use_cpu=False): 
        store_attr()
        self._cfg = self.cfg; del self.cfg;
        self.epoch,self.acc_num,self.global_step,self.eval_num,self.param_norm = 0,0,0,0,0
        self._reset_batch_dicts()
        #resume_pp_model(f"{path_checkpoints}devout-durian-172_39")
        self._setup_data_stores()
        self._setup_gradient_accumulation_variables()
        self.start_end_token_d = get_start_end_special_token_ids(self.pp_tokenizer)
        
    def train(self): 
        self._setup_wandb_run()
        ## we set num_processes=1 because we are running on 1 GPU only and we must specify the argument
        # we can't use FP16 with PEGASUS model, so full precision is needed. 
        #%lprun -f _training_function -f  get_pp_logp -f training_step -f  reward_fn -f  loss_fn -f eval_dl  notebook_launcher(_training_function, args=(pp_model, vm_model, dld_tkn, dld_raw, optimizer), num_processes=1)
        notebook_launcher(self._training_function, args=(), num_processes=1, use_fp16=False)  
    
    def _reset_batch_dicts(self): 
        # train_batch_d holds all info to write to csv, time_d has times, wandb_d has everything to log to wandb
        # there will be overlap between them. 
        self.batch_d,self.batch_time_d,self.batch_wandb_d = dict(),dict(),dict()
    
    def _setup_wandb_run(self, log_code=True): 
        """Init wandb run, set up paths, create dir for model artifacts if needed, """
        self.run, self._cfg = start_wandb_run(self._cfg, log_code=self.log_code)
        if self._cfg.wandb['log_grads']: wandb.watch(self.pp_model, log='gradients', log_freq=self._cfg.wandb['log_grads_freq'])      
        
    def _setup_data_stores(self): 
        """Setup dict `self.data_d` to store observations. Setup column names for wandb tables. """
        # Raw observation data (lists of dicts, later becomes pandas df)
        self.data_d = dict() 
        for split in self._cfg.splits + ['training_step']:   self.data_d[split] = [] 
        # Data containers and data loaders
        self.eval_epoch_df_d = dict(train=[], valid=[], test=[]) # each eval epoch dataframe appended to here 

            
    def _setup_gradient_accumulation_variables(self):
        """acc_global_l is a list of all batch sizes encountered during training. 
            """
        self.acc_global_l = self._cfg.dl_batch_sizes['train'] * self._cfg.n_train_epochs
        assert len(self.acc_global_l) ==  self._cfg.n_train_steps
        # Check if there will be leftover batches
        self._cfg.acc_leftover_batches =  self._cfg.n_train_steps % self._cfg.acc_steps
        if self._cfg.acc_leftover_batches != 0: 
            msg = f"Config set to do gradient accumulation every {self._cfg.acc_steps} batches, and there are \
            {self._cfg.n_train_steps} total training steps, so there will be {self._cfg.acc_leftover_batches} batches at \
            the end that will not be trained on."
            warnings.warn(msg)
        self._reset_acc_lists()

    def _reset_acc_lists(self):
        """call this at start and every time you call opt step"""
        # acc_current_l is a list of the batch sizes in the current accumulation batch.
        last_step = (self._cfg.n_train_steps - 1) - self._cfg.acc_leftover_batches
        if self.global_step == 0:   # at start of training
            self.acc_current_l = self.acc_global_l[self.global_step:self._cfg.acc_steps]
            assert len(self.acc_current_l) == self._cfg.acc_steps
        else: 
            self.acc_current_l = self.acc_global_l[(self.global_step+1):(self.global_step+self._cfg.acc_steps+1)]
            if self.global_step == last_step:  assert len(self.acc_current_l) == self._cfg.acc_leftover_batches
            else:                              assert len(self.acc_current_l) == self._cfg.acc_steps
        self.acc_current_n_examples = sum(self.acc_current_l)       
        
    def _eval_save_log_test_set(self): 
        """Eval on test set, convert to df, save to file, and log to wandb summary"""
        self._eval_dl(split='test')
        self.data_d["test"] = self._convert_data_d_to_df("test")
        self._set_df_colorder("test")
        self.data_d["test"].to_csv(f"{self._cfg.path_run}test.csv", index=False)
        self._add_wandb_run_summary_statistics()

    def _training_function(self): 
        self.accelerator = Accelerator(cpu=self.use_cpu)
        self._cfg.device = self.accelerator.device
        self.vm_model,self.pp_model,self.ref_pp_model,self.sts_model,self.optimizer,self.ds.dld_tkn['train'] = self.accelerator.prepare(
            self.vm_model,self.pp_model,self.ref_pp_model,self.sts_model,self.optimizer,self.ds.dld_tkn['train'])
        
        logger.debug(show_gpu(f'GPU memory usage after loading models:'))
        progress_bar = tqdm(range(self._cfg.n_train_steps))
        self.pp_model.zero_grad(set_to_none=self._cfg.zero_grad_with_none) 
        
        # initial eval (at epoch 0)
        if self.initial_eval:
            logger.info("Launching initial eval run: train")
            self._eval_dl(split='train')
            logger.info("Launching initial eval run: valid")
            self._eval_dl(split='valid')
            self._compute_and_log_eval_metrics()
        
        for self.epoch in range(1, self._cfg.n_train_epochs+1): 
            logger.info(f"Now on epoch {self.epoch} of {self._cfg.n_train_epochs}")
            if not self.pp_model.training: self.pp_model.train()
            with timecode() as time_train_one_epoch:
                for self.batch_num, (data, raw) in enumerate(zip(self.ds.dld_tkn['train'], self.ds.dld_raw['train'])): 
                    self._reset_batch_dicts()
                    self._training_step(data, raw) 
                    if self._batch_for_opt_step(): self._reset_acc_lists()
                    self.acc_num = (self.acc_num + 1) % self._cfg.acc_steps
                    self.global_step += 1
                    progress_bar.update(1) 
                    
                    
            wandb.log({'time/train_one_epoch_time': time_train_one_epoch.t,
                       'time/train_one_epoch_thoroughput': len(self.ds.dsd_tkn['train']) / time_train_one_epoch.t,
                       'epoch': self.epoch}, commit=True)

            if self._cfg.wandb['log_grads'] and self.epoch % self._cfg.wandb_log_grads_freq == 0: 
                plt = self._plot_grad_flow(self.pp_model.named_parameters())
                wandb.log({"gradient flow": wandb.Image(plt)})  # doesn't work as a non-image (i.e. plotly)
                del plt 
            #gc.collect() 
            #torch.cuda.empty_cache()

            if self._cfg.save_model_while_training and (self.epoch + 1) % self._cfg.save_model_freq == 0:  save_model(epoch)

            # Evaluation loop
            if self.epoch % self._cfg.eval_freq == 0: 
                self.eval_num += 1
                with timecode() as time_eval_train:
                    self._eval_dl(split='train') 
                with timecode() as time_eval_valid:
                    self._eval_dl(split='valid')
                with timecode() as time_eval_compute_metrics: 
                    self._compute_and_log_eval_metrics()
                with timecode() as time_eval_gc_collect:
                    gc.collect() 
                with timecode() as time_eval_empty_cache:
                    torch.cuda.empty_cache()
                wandb.log({'time/eval_train_time': time_eval_train.t, 'time/eval_valid_time': time_eval_valid.t,
                           'time/eval_train_thoroughput': len(self.ds.dsd_tkn['train']) / time_eval_train.t,
                           'time/eval_valid_thoroughput': len(self.ds.dsd_tkn['valid']) / time_eval_valid.t, 
                           'time/eval_gc_collect': time_eval_gc_collect.t, 
                           'time/eval_empty_cache': time_eval_empty_cache.t,
                           'time/eval_compute_metrics': time_eval_compute_metrics.t,
                           'epoch': self.epoch}, commit=True)
        
        self._eval_save_log_test_set()
    
    def _training_step(self, data, raw): 
        """Forward pass, loss function, backwards pass, parameter update (with gradient accumulation optional), 
        recording results, wandb logging. 
        """
        if not self.pp_model.training: self.pp_model.train()
        if not self.vm_model.training: self.vm_model.train()
        with timecode() as self.batch_time_d['time_generate_pp']:
            pp_output, pp_l = self._pp_model_forward(data)
        
        logger.debug(show_gpu(f'TRAIN, epoch {self.epoch}, batch {self.batch_num}, GPU memory usage after forward pass: '))
        
        # autocast is used by accelerate to allow mixed-precision loss functions.
        # drop it if we deprecate fp16 support (because it isn't supported for models like PEGASUS)
        #with self.accelerator.autocast():
        
        with timecode() as self.batch_time_d['time_loss_fn']:
            loss_batch = self._loss_fn(data, raw, pp_output, pp_l)

        with timecode() as self.batch_time_d['time_backwards']:
            self.accelerator.backward(loss_batch) 

        with timecode() as self.batch_time_d['time_calc_gradient_norm']: 
            self.grad_norm =  self._get_gradient_update_norm()
        
        logger.debug(show_gpu(f'TRAIN, epoch {self.epoch}, batch {self.batch_num}, GPU memory usage after backwards pass: '))
        with timecode() as self.batch_time_d['time_opt_step_and_calc_param_norm']:
            if self._batch_for_opt_step():  
                self._opt_step_and_calc_param_norm()
            else: 
                self.param_norm = 0       
                with timecode() as self.batch_time_d['time_opt_step']: pass
            
        self._prepare_train_batch_d(raw, data, pp_l)
        self.data_d['training_step'].append(self.batch_d)
        self._wandb_log_training_step()
    
    def _opt_step_and_calc_param_norm(self): 
        ## record initial parameters
        params_all = [o.detach() for o in self.pp_model.parameters() if o[1].requires_grad]
        params_initial = [ p.clone() for p in params_all]
        # step + zero grad
        with timecode() as self.batch_time_d['time_opt_step']: 
            self.optimizer.step()
            self.pp_model.zero_grad(set_to_none=self._cfg.zero_grad_with_none)
        # record norm
        self.param_norm = 0 
        for (p_init, p_new) in zip(params_initial, params_all): 
            self.param_norm += (p_new - p_init).data.norm(2).item() ** 2
        self.param_norm = self.grad_norm ** 0.5 
    
    def _batch_for_opt_step(self): return self.acc_num == (self._cfg.acc_steps - 1)
    
    def _add_batch_vars_to_batch_d(self, raw, data, pp_l): 
        # Add basics. (results are already added elsewhere)
        self.batch_d = merge_dicts(self.batch_d, { 'idx': raw['idx'],
            'epoch': self.epoch, 'batch_num': self.batch_num, 'global_step': self.global_step,
            'acc_num': self.acc_num, "acc_batch_n_examples": self.acc_current_n_examples, 
            "orig_l": raw['text'], 
            "orig_label": data['label'].cpu().tolist(), 
            "orig_truelabel_probs": data['orig_truelabel_probs'].cpu().tolist(),
            'orig_length': self.orig_length, 'orig_batch_size': self.orig_batch_size, 
            "pp_l": pp_l, 'pp_length': self.pp_length, 'pp_batch_size': self.pp_batch_size
        })
        
    def _prepare_train_batch_d(self, raw, data, pp_l): 
        self._add_batch_vars_to_batch_d(raw, data, pp_l)
        # Add times (only for training, not eval)
        for k, v in self.batch_time_d.items(): self.batch_time_d[k] = v.t  # extract time from timecode object
        self.batch_d = merge_dicts(self.batch_d, self.batch_time_d)
    
    def _wandb_log_training_step(self): 
        self.batch_wandb_d = merge_dicts(self.batch_wandb_d, {
            'vm_scores_hist':       Histogram(self.batch_d['vm_score']), 
            'vm_scores_mean':       np.mean(  self.batch_d['vm_score']),
            'sts_scores_hist':      Histogram(self.batch_d['sts_score']),
            'sts_scores_mean':      np.mean(  self.batch_d['sts_score']), 
            'rewards_hist':         Histogram(self.batch_d['reward']),
            'rewards_mean':         np.mean(  self.batch_d['reward']), 
            'pp_logp_hist':         Histogram(self.batch_d['pp_logp']),
            'pp_logp_mean':         np.mean(  self.batch_d['pp_logp']),
            'ref_logp_hist':        Histogram(self.batch_d['ref_logp']),
            'ref_logp_mean':        np.mean(  self.batch_d['ref_logp']),
            'kl_div_hist' :         Histogram(self.batch_d['kl_div']),
            'kl_div_mean':          np.mean(  self.batch_d['kl_div']),
            'reward_penalty_hist':               Histogram(self.batch_d['reward_penalty']),
            'reward_penalty_mean':               np.mean(  self.batch_d['reward_penalty']), 
            'reward_with_penalty_hist':          Histogram(self.batch_d['reward_with_penalty']),
            'reward_with_penalty_mean':          np.mean(  self.batch_d['reward_with_penalty']),
            'loss_hist'   :         Histogram(self.batch_d['loss']), 
            'pp_letter_diff_hist':     Histogram(self.batch_d['pp_letter_diff']),
            'pp_letter_diff_mean':     np.mean(  self.batch_d['pp_letter_diff']), 
            'pp_letter_percent_hist':  Histogram(self.batch_d['pp_letter_percent']),
            'pp_letter_percent_mean':  np.mean(  self.batch_d['pp_letter_percent']),
            'contradiction_score_hist':     Histogram(self.batch_d['contradiction_score']),
            'contradiction_score_mean':     np.mean(  self.batch_d['contradiction_score']), 
            'acc_batch_sizes':      Histogram(self.acc_current_l), 
            "gradient_norm":        self.grad_norm, 
            "parameter_norm":       self.param_norm
        })
        self.batch_wandb_d = merge_dicts(self.batch_wandb_d, self.batch_d)
        not_for_wandb_keys = ['orig_l', 'orig_label','orig_truelabel_probs', 'pp_l', 'loss', 'pp_logp','ref_logp', 'kl_div', 'reward_with_penalty', 'reward_penalty',
                              'reward', 'sts_score', 'vm_score', 'pp_letter_diff', 'pp_letter_percent','contradiction_score',
                              'pp_predclass_probs', 'label_flip', 'pp_predclass', 'pp_truelabel_probs']
        for k in not_for_wandb_keys:  self.batch_wandb_d.pop(k, None)
        wandb.log(self.batch_wandb_d, commit=True)
        
    def _convert_data_d_to_df(self, data_d_key): 
        df = pd.DataFrame(self.data_d[data_d_key])
        ### Check all lists have the same number of elements in their row 
        # last batch will have different number of elements to the batch size
        nonscalar_cols = df.columns[[o == np.dtype('object') for o in df.head(1).dtypes]].tolist()
        df_nonscalar_cols = df[nonscalar_cols]
        # sometimes if we have one element in the last batch, the tolist() function returns a scalar instead of a list 
        # so we handle that case here
        def scalar2list(x): return x if type(x) == list else [x]
        if len(df_nonscalar_cols.iloc[-1]["idx"]) == 1: df_nonscalar_cols.iloc[-1] = df_nonscalar_cols.iloc[-1].apply(scalar2list) 
        df_lengths = df_nonscalar_cols.applymap(len)
        assert df_lengths.eq(df_lengths.iloc[:,0], axis=0).all(None)
        
        ### Put in dataframes
        # expand lists and broadcast scalars
        scalar_cols = df.columns[[o != np.dtype('object') for o in df.head(1).dtypes]].tolist()
        df_expanded = unpack_nested_lists_in_df(df, scalar_cols)
        # check shape of new dataframe is correct 
        if data_d_key == "training_step": 
            if self.epoch == 0: 
                df_shape = (self._cfg.ds_length["train"],                       df.shape[1])
            else: 
                df_shape = (self._cfg.ds_length["train"] * self._cfg.eval_freq, df.shape[1])
        elif data_d_key in ["train", "valid", "test"]: 
                 df_shape = (self._cfg.ds_length[data_d_key],                   df.shape[1])
        assert df_expanded.shape == df_shape
        return df_expanded
    
    def _pp_model_forward(self, data): 
        pp_output, pp_l = self._get_paraphrases(data['input_ids'], data['attention_mask'])
       # self._assert_start_and_end_tokens_are_correct(orig_ids=data['input_ids'], pp_ids=pp_output.sequences)
        # Keep the below line here because then both training and eval can access it
        self._update_batch_size_and_length_variables(orig_ids=data['input_ids'], pp_ids=pp_output.sequences)
        return pp_output, pp_l
    
    def _assert_start_and_end_tokens_are_correct(self, orig_ids, pp_ids):
        """Make sure input sequences (orig) and output sequences (pp) start and end with the 
        right special tokens (depends on tokenizer)"""
        # Input
        if self.start_end_token_d['input_start_id'] is not None: 
            assert torch.all(orig_ids[:,0] == self.start_end_token_d['input_start_id'])
        # can probs rewrite this to make it nicer but it's fine for now
        assert torch.all(torch.logical_or(orig_ids[:,-1] == self.start_end_token_d['input_end_id'][0], 
                                          orig_ids[:,-1] == self.start_end_token_d['input_end_id'][1]))

        # Output
        assert torch.all(pp_ids[:,0] == self.start_end_token_d['output_start_id'])
        assert torch.all(torch.logical_or(pp_ids[:,-1] == self.start_end_token_d['output_end_id'][0], 
                                          pp_ids[:,-1] == self.start_end_token_d['output_end_id'][1]))
        
    def _update_batch_size_and_length_variables(self, orig_ids, pp_ids): 
        # Update variables
        # for greedy search self.pp_length is equal to self.orig_batch_size but this won't be for beam search
        self.orig_batch_size     = orig_ids.shape[0]
        self.orig_length         = orig_ids.shape[1]
        self.pp_batch_size       = pp_ids.shape[0]
        self.pp_length           = pp_ids.shape[1] 
    
    def _get_paraphrases(self, orig_ids, attention_mask):
        """Wrapper for generating paraphrases (pp's)."""
        pp_output = self.pp_model.generate_with_grad(input_ids=orig_ids, attention_mask=attention_mask, num_return_sequences=1, num_beams=1,
                                                     **self._cfg.pp, return_dict_in_generate=True, output_scores=True, remove_invalid_values=False, 
                                                     pad_token_id = self.pp_tokenizer.pad_token_id,eos_token_id = self.pp_tokenizer.eos_token_id)
        pp_l = self.pp_tokenizer.batch_decode(pp_output.sequences, skip_special_tokens=True)
        return pp_output, pp_l
    
    def _loss_fn(self, data, raw, pp_output, pp_l): 
        with timecode() as self.batch_time_d['time_reward_fn']:
            reward = self._reward_fn(data, raw, pp_l)

        with timecode() as self.batch_time_d['time_pp_logp']:
            pp_logp = self._get_pp_logp(pp_output)

        with timecode() as self.batch_time_d['time_ref_logprobs']: 
            ref_logp = self._get_ref_logprobs(orig_ids=data['input_ids'], pp_ids=pp_output.sequences)
            
        kl_div =  torch.clip(pp_logp - ref_logp, min=0)  # sometimes this is negative, not sure why. 
        with timecode() as self.batch_time_d['time_loss_fn_loss_calc']:
            if   self._cfg.reward_penalty_type == "kl_div": 
                reward_penalty = - (self._cfg.kl_coef       * kl_div)       # KL div is positive, this term is negative
            elif self._cfg.reward_penalty_type == "ref_logp": 
                reward_penalty =    self._cfg.ref_logp_coef * ref_logp      # ref_logp is negative, this term is negative
                
            reward_with_penalty = torch.clip(reward + reward_penalty, min=0) 
            loss       = -reward_with_penalty * pp_logp
            loss_sum   = torch.sum(loss)  # we scale it later
            loss_batch = loss_sum / self.acc_current_n_examples  # for gradient accumulation     
  
        self.batch_d['pp_logp']     =                      pp_logp.detach().cpu().tolist()
        self.batch_d['ref_logp']    =                     ref_logp.detach().cpu().tolist()
        self.batch_d['kl_div']      =                 kl_div.detach().cpu().tolist()
        self.batch_d['reward_penalty'] =            reward_penalty.detach().cpu().tolist()
        self.batch_d['reward_with_penalty'] =  reward_with_penalty.detach().cpu().tolist()
        self.batch_d['loss']       =                          loss.detach().cpu().tolist()
        self.batch_d['loss_sum']   =                      loss_sum.detach().cpu().tolist()
        self.batch_d['loss_batch']   =                  loss_batch.detach().cpu().tolist()
        return loss_batch
        
    def _get_vm_scores(self, pp_l, labels, orig_truelabel_probs): 
        pp_probs = get_vm_probs(pp_l, self._cfg, self.vm_tokenizer, self.vm_model, return_predclass=False)
        pp_predclass = torch.argmax(pp_probs, axis=1)
        pp_truelabel_probs   = torch.gather(pp_probs, 1, labels[:,None]).squeeze()
        pp_predclass_probs   = torch.gather(pp_probs, 1, pp_predclass[ :,None]).squeeze()
        label_flip = ((pp_predclass != labels) * 1)
        vm_scores = (orig_truelabel_probs - pp_truelabel_probs)
        return dict(pp_truelabel_probs=pp_truelabel_probs, pp_predclass=pp_predclass, pp_predclass_probs=pp_predclass_probs, vm_scores=vm_scores, label_flip=label_flip)
        
    def _get_pp_letter_diff(self, pp_l, orig_n_letters): 
        pp_n_letters = np.array([len(o) for o in pp_l])
        orig_letters = np.array(orig_n_letters)
        pp_letter_diff    = orig_n_letters - pp_n_letters
        pp_letter_percent = pp_n_letters / orig_n_letters 
        return dict(pp_letter_diff=pp_letter_diff, pp_letter_percent=pp_letter_percent)
    
    def _get_contradiction_scores(self, orig_l, pp_l):
        contradiction_scores = get_nli_probs(orig_l, pp_l, self._cfg, self.nli_tokenizer, self.nli_model)[:,  self._cfg.contra_label]
        return contradiction_scores
    
    def _get_sts_scores_one_to_many(self, pp_l, orig_sts_embeddings): 
        """Calculate STS scores when there is one orig and a list of pp_l"""
        orig_sts_embeddings = orig_sts_embeddings.to(self._cfg.device)
        pp_sts_embeddings = self.sts_model.encode(pp_l, convert_to_tensor=True, device=self._cfg.device, show_progress_bar=False)
        sts_scores = pytorch_cos_sim(orig_sts_embeddings, pp_sts_embeddings).cpu().tolist()
        return sts_scores 
    
    def _is_valid_pp(self, sts_score, pp_letter_diff, contradiction_score): 
        if sts_score           < self._cfg.sts_threshold:                              return False
        if contradiction_score > self._cfg.contradiction_threshold:                    return False  
        if pp_letter_diff >   self._cfg.pp_letter_diff_threshold:                      return False 
        if pp_letter_diff < - self._cfg.pp_letter_diff_threshold:                      return False
        return True
    
    def _get_reward(self, vm_scores, sts_scores, pp_letter_diff, contradiction_scores): 
        def reward_fn_contradiction_and_letter_diff(vm_score, sts_score, pp_letter_diff, contradiction_score): 
            if not self._is_valid_pp(sts_score, pp_letter_diff, contradiction_score): return 0
            reward = self._cfg.reward_base + vm_score * self._cfg.reward_vm_multiplier  
            return min(max(self._cfg.reward_clip_min, reward), self._cfg.reward_clip_max)
        
        def calc_reward(vm_scores, sts_scores, pp_letter_diff, contradiction_scores):
            if self._cfg.reward_fn == "reward_fn_contradiction_and_letter_diff": reward_fn = reward_fn_contradiction_and_letter_diff
            return torch.tensor([reward_fn(vm, sts, ldiff, contra) for vm,sts,ldiff,contra in zip(vm_scores, sts_scores, pp_letter_diff, contradiction_scores)],
                                device=self._cfg.device)
        rewards = calc_reward(vm_scores, sts_scores, pp_letter_diff, contradiction_scores)
        return rewards
    
    
    def _reward_fn(self, data, raw, pp_l): 
        """Used for training, need 1-1 """
        # Victim model probability differences between orig and pp
        # #TODO: update this whole function
        with timecode() as self.batch_time_d['time_vm_scores']:
            pp_probs = get_vm_probs(pp_l, self._cfg, self.vm_tokenizer, self.vm_model, return_predclass=False)
            pp_predclass = torch.argmax(pp_probs, axis=1)
            pp_truelabel_probs   = torch.gather(pp_probs, 1, data['label'][:,None]).squeeze()
            pp_predclass_probs   = torch.gather(pp_probs, 1, pp_predclass[ :,None]).squeeze()
            label_flip = ((pp_predclass != data['label']) * 1)
            vm_scores = (data['orig_truelabel_probs'] - pp_truelabel_probs)
            
        # STS scores
        with timecode() as self.batch_time_d['time_sts_scores']:
            pp_embeddings  = self.sts_model.encode(pp_l, batch_size=len(raw), convert_to_tensor=True, show_progress_bar=False, device=self._cfg.device)
            # This returns a cosine similarity matrix, of which we just want the diagonal
            sts_scores = pytorch_cos_sim(data['orig_sts_embeddings'], pp_embeddings).diagonal()  
        
        with timecode() as self.batch_time_d['time_pp_letter_diff']:
            pp_letters = np.array([len(o) for o in pp_l])
            orig_letters = data['n_letters'].cpu().numpy()
            pp_letter_diff    = orig_letters - pp_letters
            pp_letter_percent = pp_letters / orig_letters 
            
            
        with timecode() as self.batch_time_d['time_contradiction_scores']: 
            contradiction_scores = get_nli_probs(raw['text'], pp_l, self._cfg, self.nli_tokenizer, self.nli_model)[:,  self._cfg.contra_label]

        # Reward calculation
        
        def reward_fn_contradiction_and_letter_diff(vm_score, sts_score, pp_letter_diff, contradiction_score): 
            if sts_score           < self._cfg.sts_threshold:                              return 0
            if contradiction_score > self._cfg.contradiction_threshold:                    return 0  
            if pp_letter_diff >   self._cfg.pp_letter_diff_threshold:                      return 0 
            if pp_letter_diff < - self._cfg.pp_letter_diff_threshold:                      return 0 
            reward = self._cfg.reward_base + vm_score * self._cfg.reward_vm_multiplier  
            return min(max(self._cfg.reward_clip_min, reward), self._cfg.reward_clip_max)
        
        
        def calc_reward(vm_scores, sts_scores, pp_letter_diff, contradiction_scores):
            if self._cfg.reward_fn == "reward_fn_contradiction_and_letter_diff": reward_fn = reward_fn_contradiction_and_letter_diff
            return torch.tensor([reward_fn(vm, sts, ldiff, contra) for vm,sts,ldiff,contra in zip(vm_scores, sts_scores, pp_letter_diff, contradiction_scores)],
                                device=self._cfg.device)
        
        rewards = calc_reward(vm_scores, sts_scores, pp_letter_diff, contradiction_scores)
        self.batch_d['pp_truelabel_probs']  = pp_truelabel_probs.detach().cpu().tolist()
        self.batch_d['pp_predclass']        = pp_predclass.detach().cpu().tolist()
        self.batch_d['pp_predclass_probs']  = pp_predclass_probs.detach().cpu().tolist()
        self.batch_d['label_flip']          = label_flip.detach().cpu().tolist()
        self.batch_d['label_flip_fraction'] = np.mean(self.batch_d['label_flip'])
        self.batch_d['reward']              = rewards.detach().cpu().tolist()
        self.batch_d['vm_score']            = vm_scores.detach().cpu().tolist()
        self.batch_d['sts_score']           = sts_scores.detach().cpu().tolist()
        self.batch_d['pp_letter_diff']      = pp_letter_diff.tolist()
        self.batch_d['pp_letter_percent']   = pp_letter_percent.tolist()
        self.batch_d['contradiction_score'] = contradiction_scores.cpu().tolist()
        return rewards
         
    def _get_pp_logp(self, pp_output): 
        """log(p(pp|orig)) basically.
        works for greedy search, will need tweaking for other types probably"""
        ### We want to align tokens with token probabilities. The first token is given at the start 
        # and has no probability attached to it, so we remove it. 
        seq_without_first_tkn = pp_output.sequences[:, 1:]
        assert seq_without_first_tkn.shape == torch.Size([self.orig_batch_size, self.pp_length - 1])

        ### Convert from tuple of scores to one big tensor of scores 
        scores_stacked = torch.stack(pp_output.scores, 1)
        ### TESTS 
        # We check shape and that there is no +inf or nan in scores. 
        # Scores can have -inf in them - see explanation in `exploring_generation`.  
        assert scores_stacked.shape == torch.Size([self.orig_batch_size, (self.pp_length - 1), self._cfg.vocab_size])
        assert torch.all(~torch.isnan(scores_stacked))
        assert torch.all(~torch.isposinf(scores_stacked))
        
#         # Rough check that every time the eos token occurs before min_length it is -inf for all elements in batch
#         # We do min_length - 1 because sequences are allowed to have length min_length so that idx 
#         # shouldn't be set to -inf
#         # Not a 100% test but very likely to identify
#         idx_neginf = torch.nonzero(torch.isneginf(scores_stacked))
#         assert len(idx_neginf[idx_neginf[:,2] == self.pp_tokenizer.eos_token_id, :]) == \
#                   (self._cfg.pp["min_length"] -1) * self.orig_batch_size  
#         del idx_neginf

        ### Take log softmax of scores and then extract those that correspond 
        # to the generated sequences    
        scores_log_softmax = scores_stacked.log_softmax(2)
        seq_token_log_probs = torch.gather(scores_log_softmax,2,seq_without_first_tkn[:,:,None]).squeeze(-1)
        ### TESTS 
        # -inf is possible in scores_log_softmax and seq_token_log_probs before the attention mask is added. 
        assert torch.all(~torch.isnan(   scores_log_softmax))
        assert torch.all(~torch.isposinf(scores_log_softmax))
        self._check_scores_log_softmax_sums(scores_log_softmax)
        # probs should be 1-1 with the filtered tkns: check shape to confirm
        assert seq_token_log_probs.shape == seq_without_first_tkn.shape  
        # Check that the last token probability corresponds to a possible end token
        # this has to be tested before the attention mask is multiplied with it because if the 
        # padding token is 0 then this will be 0 too (and not the same as scores_log_softmax)
     #   output_end_ids = self.start_end_token_d['output_end_id']
     #   assert all([o in scores_log_softmax[:, -1, output_end_ids] for o in seq_token_log_probs[:,-1]])
     #   del output_end_ids
        ## THIS ONE IS LONG - a test rather than assert 
        # check_seq_token_log_prob_values_are_correct(seq_without_first_tkn, scores_log_softmax, 
        #                                             seq_token_log_probs) 

        ### Generate attention mask to identify padding tokens. Then apply it to the 
        # sequence probabilities so that we don't consider probability of padding tokens 
        # when getting sequence probabilities. 
        # Also replace the -inf values in seq_token_log_probs with a large negative number because if we 
        # leave them in we end up with nan's introduced after multiplying with attention_mask, 
        # since  -inf * 0 = nan 
        attention_mask = self.pp_model._prepare_attention_mask_for_generation(
            seq_without_first_tkn, self.pp_tokenizer.pad_token_id, self.pp_tokenizer.eos_token_id
        )
        seq_token_log_probs = torch.nan_to_num(seq_token_log_probs, nan=None, posinf=None, neginf=-20)
        seq_token_log_probs = seq_token_log_probs * attention_mask
        ### TESTS
        assert seq_token_log_probs.shape == attention_mask.shape == seq_token_log_probs.shape
        # check attention mask only has 0 for padding tokens and not eos tokens or anything else
        assert all(seq_without_first_tkn[attention_mask == 0] == self.pp_tokenizer.pad_token_id)
        check_no_nans_or_infs(seq_token_log_probs)
        # check that we aren't picking extrememly rare tokens
        # assert torch.all(seq_token_log_probs  > -10)  

        ### Get sequence probabilities by summing up token log probabilities 
        seq_log_prob = seq_token_log_probs.sum(-1)
        ## TESTS 
        assert seq_log_prob.shape == torch.Size([self.pp_batch_size])
        check_no_nans_or_infs(seq_log_prob)
        
        # normalise for length
        logprobs_normalised = seq_log_prob / attention_mask.sum(1)  # normalise for length of generated sequence
        
        if self.pp_model.training:  # don't bother logging or calculate entropy, token_probs in eval mode
            if self._cfg.wandb['log_token_entropy']:
                with timecode() as self.batch_time_d['time_log_entropy']:
                    self.batch_wandb_d['ent_hist'] = self._get_entropy_hist(scores_stacked, attention_mask) 
            if self._cfg.wandb['log_token_probabilities']: 
                with timecode() as self.batch_time_d['time_log_token_probabilities']:
                    self.batch_wandb_d = merge_dicts(self.batch_wandb_d, 
                        self._get_token_probability_metrics(scores_log_softmax, attention_mask, k=3))
        return logprobs_normalised
    
    def _get_ref_logprobs(self, orig_ids, pp_ids): 
     #   orig_input_ids = self.pp_tokenizer(orig_l, return_tensors='pt', padding=True, truncation=True).input_ids
       # pp_input_ids   = self.pp_tokenizer(pp_l,   return_tensors='pt', padding=True, truncation=True).input_ids
        decoder_start_token_ids = torch.tensor([self.ref_pp_model.config.decoder_start_token_id], device=self._cfg.device).repeat(self.orig_batch_size, 1)
        pp_ids = torch.cat([decoder_start_token_ids, pp_ids], 1)
        logprobs = []
        for i in range(pp_ids.shape[1] - 1): 
            decoder_input_ids = pp_ids[:, 0:(i+1)]
            outputs = self.ref_pp_model(input_ids=orig_ids, decoder_input_ids=decoder_input_ids)
            token_logprobs = outputs.logits[:,i,:].log_softmax(1)
            pp_next_token_ids = pp_ids[:,i+1].unsqueeze(-1)
            pp_next_token_logprobs = torch.gather(token_logprobs, 1, pp_next_token_ids).detach().squeeze(-1)
            logprobs.append(pp_next_token_logprobs)
        logprobs = torch.stack(logprobs, 1)   
        attention_mask = self.ref_pp_model._prepare_attention_mask_for_generation(pp_ids[:,1:],
                self.pp_tokenizer.pad_token_id, self.pp_tokenizer.eos_token_id)
        logprobs = logprobs * attention_mask        
        logprobs_sum = logprobs.sum(1)
        logprobs_normalised = logprobs_sum / attention_mask.sum(1)  # normalise for length of generated sequence
        return logprobs_normalised
   
    def _check_scores_log_softmax_sums(self, scores_log_softmax):
        sums = scores_log_softmax.exp().sum(2)
        # check that the axes is right
        # we want to sum over token probabilities at each generation step, so we 
        # should end up with a shape [self.orig_batch_size, self.pp_length]
        assert sums.shape[0] == self.orig_batch_size  
        assert sums.shape[1] == self.pp_length - 1
        # check that they sum to 1 along the self.pp_length axis (or close enough at least) 
        assert torch.allclose(sums, torch.ones(sums.size(), device=self._cfg.device), atol = 5e-2)

    def _check_seq_token_log_prob_values_are_correct(self, seq_without_first_tkn, scores_log_softmax, seq_token_log_probs): 
        """Just enumerates and checks values
        Quite slow for large batches so run as a test rather than an assert in every batch. 
        """
        l = []
        for i_ex in range(self.orig_batch_size):
            for i_step in range(self.pp_length - 1):
                i_tkn = seq_without_first_tkn[i_ex][i_step].item()
                l.append(scores_log_softmax[i_ex,i_step, i_tkn] == seq_token_log_probs[i_ex,i_step])
        assert all(l)    
    
    def _get_entropy_hist(self, scores_stacked, attention_mask): 
        ent = Categorical(logits = scores_stacked).entropy().detach()
        assert ent.shape == attention_mask.shape == torch.Size([self.pp_batch_size, self.pp_length - 1])
        ent = ent * attention_mask  # stop values after eos token from contributing to ent score 
        # first remove structure (otherwise we have ragged arrays), then remove corresponding attention mask values
        # we can't just filter by ent[ent != 0] because we might have zero tokens during the sequence
        att_flat= attention_mask.flatten()
        indices = torch.nonzero(att_flat)
        ent_flat = ent.flatten()[indices].flatten()
        assert ent_flat.shape[0] == (torch.sum(att_flat)*1).item()
        # check everything we filter out is zero 
        torch.isclose(ent.flatten()[torch.nonzero(~(att_flat > 0))].sum(), torch.tensor(0.), 1e-3)
        return Histogram(ent_flat.detach().cpu().tolist())

    def _get_token_probability_metrics(self, scores_log_softmax, attention_mask, k=3): 
        token_prob_d = dict()
        tkn_kmaxprob, _ = torch.topk(scores_log_softmax, largest=True, k=k, dim=2)
        tkn_kmaxprob = tkn_kmaxprob.detach()  
        tkn_kmaxprob = torch.nan_to_num(tkn_kmaxprob, nan=None, posinf=None, neginf=-20)
        assert tkn_kmaxprob.shape == torch.Size([self.pp_batch_size, self.pp_length - 1, k])

        # % of first prob over 0.9, 0.75, 0.5, 0.3, 0.1
        top_probs = tkn_kmaxprob[:,:,0].exp()
        top_probs = (top_probs * attention_mask).flatten()
        top_probs = top_probs[top_probs != 0]
        prob_threshold_l = [0.99, 0.975, 0.95, 0.90, 0.75, 0.5, 0.3, 0.1]
        for p in prob_threshold_l: 
            token_prob_d[f"top_token_prob_over_{str(p)}"] = (torch.sum(top_probs > p) / top_probs.shape[0]).item()

        # avg + median + lower + upper quartile of first, second, third choice probs
        tkn_kmaxprob_mask = tkn_kmaxprob * attention_mask[:,:,None]  # broadcasting over kth dim
        for i in range(k): 
            probs = tkn_kmaxprob_mask[:,:, i].flatten()
            probs = probs[probs != 0]
            token_prob_d[f"rank_{i+1}_histogram"] = Histogram(probs.detach().cpu().tolist())
            token_prob_d[f"rank_{i+1}_token_prob_mean"] = probs.mean().item()
            token_prob_d[f"rank_{i+1}_token_prob_median"] = probs.median().item()
            token_prob_d[f"rank_{i+1}_token_prob_0.25_quantile"] = probs.quantile(0.25).item()
            token_prob_d[f"rank_{i+1}_token_prob_0.75_quantile"] = probs.quantile(0.75).item()

        # tokens over probs above 0.1, 0.01, 0.001, 0.0001, 1/vocab_size prob 
        allprobs = (scores_log_softmax.detach().exp() * attention_mask[:,:,None]).flatten()
        allprobs = allprobs[allprobs != 0]
        for p in [1e-5, 1e-6, 1e-7, 1e-8, 1e-9]: 
            token_prob_d[f"%_of_tokens_above_prob_{p}"] =  (torch.sum(allprobs > p) / allprobs.shape[0]).item()
        token_prob_d[f"%_of_tokens_above_prob_1/vocab_size"] = \
            (torch.sum(allprobs > (1/self._cfg.vocab_size)) / allprobs.shape[0]).item()
        return token_prob_d
    
    def _eval_dl(self, split): 
        """Get evaluation metrics for a dataloader"""
        # Put models in eval mode and do the forward pass 
        # Current logic: push all batches together into one big list.   
        self._reset_batch_dicts()
        if self.pp_model.training: self.pp_model.eval()
        if self.vm_model.training: self.vm_model.eval()
        # The "train_eval" dataloader is the same as train but a bigger batch size and explicitly no shuffling
        dl_key = "train_eval" if split == "train" else split
        dl_raw = self.ds.dld_raw[dl_key]
        dl_tkn = self.ds.dld_tkn[dl_key]
        with torch.no_grad(): 
            for self.batch_num, (data, raw) in enumerate(zip(dl_tkn, dl_raw)):
                logger.debug(f"EVAL: {split} with dl_key {dl_key}")
                logger.debug(f"Elements in data_d[{split}]: {len(self.data_d[split])}")
                logger.debug(show_gpu(f'EVAL, epoch {self.epoch}, batch {self.batch_num}, GPU memory usage after loading data: '))
                assert data['input_ids'].shape[0] == len(raw['text_with_prefix'])
                self._reset_batch_dicts()
                assert len(self.batch_d) == len(self.batch_time_d) == len(self.batch_wandb_d) == 0 
                for k, v in data.items():
                    # Eval data isn't loaded on GPU by default unlike train data. This is because train dataloader goes 
                    # through accelerator `prepare` function, but eval dataloaders don't. So here we load the data onto GPU 
                    if data[k].device != self._cfg.device: data[k] = data[k].to(self._cfg.device)
                pp_output, pp_l = self._pp_model_forward(data)
                _ = self._loss_fn(data, raw, pp_output, pp_l)
                self._add_batch_vars_to_batch_d(raw, data, pp_l)
                self.data_d[split].append(self.batch_d)
                logger.debug(show_gpu(f'EVAL, epoch {self.epoch}, batch {self.batch_num}, GPU memory usage after loss_fn pass: '))
                
    def eval_ref_model(self, split): 
        notebook_launcher(self._eval_function, args=(split, True), num_processes=1, use_fp16=False)
        
    def _eval_function(self, split, eval_ref_model=False): 
        ## Setup
        model = self.ref_pp_model if eval_ref_model else self.pp_model
        if model.training:             model.eval()
        if self.vm_model.training:     self.vm_model.eval()
        if eval_ref_model: 
            self.accelerator_eval = Accelerator(cpu=self.use_cpu)
            self._cfg.device = self.accelerator_eval.device
            self.vm_model,model,self.sts_model = self.accelerator.prepare(self.vm_model,model,self.sts_model)
        eval_batch_results = list()  # each eval batch appended to here, list of dicts
        dl_key = "train_eval" if split == "train" else split
        dl_raw = ds.dld_raw[dl_key]
        dl_tkn = ds.dld_tkn[dl_key]

        ## Loop through batches in eval set
        for eval_batch_num, (data, raw) in enumerate(zip(dl_tkn, dl_raw)):
            pp_output = model.generate(input_ids=data['input_ids'].to(self._cfg.device), attention_mask=data['attention_mask'].to(self._cfg.device), 
                                          **self._cfg.eval_gen_params,   remove_invalid_values=False, 
                                          pad_token_id = self.pp_tokenizer.pad_token_id,eos_token_id = self.pp_tokenizer.eos_token_id)
            pp_l = self.pp_tokenizer.batch_decode(pp_output, skip_special_tokens=True)
            pp_l_nested = [pp_l[i:i+self._cfg.n_eval_seq] for i in range(0, len(pp_l), self._cfg.n_eval_seq)]  # put paraphrases in nested lists 
            assert all([len(l) == self._cfg.n_eval_seq for l in pp_l_nested])  # make sure we generate the same number of paraphrases for each
            eval_batch_results.append({'idx': raw['idx'], 'orig': raw['text'], 'pp_l':pp_l_nested, 'orig_n_letters': data['n_letters'].tolist(), 
                                  'label': raw['label'], 'orig_truelabel_probs': data['orig_truelabel_probs'].tolist(), 'orig_sts_embeddings': data['orig_sts_embeddings'] })

        ## Convert eval batches to dataframes and create paraphrase identifier `pp_idx`
        df = pd.DataFrame(eval_batch_results)    
        df = df.apply(pd.Series.explode).reset_index(drop=True)  # This dataframe has one row per original example
        def get_pp_idx(row): return ["orig_" + str(row['idx']) + "-epoch_" + str(epoch) +  "-pp_" +  str(pp_i) for pp_i in range(1, len(row['pp_l'])+1)]
        df['pp_idx'] = df.apply(get_pp_idx, axis=1)

        ## Create seperate dataframe for sts scores and expand original dataframe
        df_sts = df[['pp_idx', 'pp_l', 'orig_sts_embeddings']] 
        df1 = df.drop(columns='orig_sts_embeddings')
        scalar_cols = [o for o in df1.columns if o not in ['pp_l', 'pp_idx']]
        df_expanded = unpack_nested_lists_in_df(df1, scalar_cols=scalar_cols) # This dataframe has one row per paraphrase

        ## Add vm_scores, sts_scores, pp_letter_diff, contradiction scores
        ds_expanded = Dataset.from_pandas(df_expanded)
        def add_vm_scores_eval(batch): 
            output = self._get_vm_scores(pp_l=batch['pp_l'], labels=torch.tensor(batch['label'], device = self._cfg.device), 
                                            orig_truelabel_probs=torch.tensor(batch['orig_truelabel_probs'], device=self._cfg.device))
            for k, v in output.items(): batch[k] = v.cpu().tolist() 
            return batch
        def add_pp_letter_diff(batch): 
            output = self._get_pp_letter_diff(pp_l=batch['pp_l'], orig_n_letters=batch['orig_n_letters'])
            for k, v in output.items(): batch[k] = v.tolist() 
            return batch
        def add_contradiction_score(batch): 
            batch['contradiction_scores'] = self._get_contradiction_scores(orig_l=batch['orig'], pp_l=batch['pp_l']).cpu().tolist()
            return batch
        ds_expanded = ds_expanded.map(add_vm_scores_eval,        batched=True)
        ds_expanded = ds_expanded.map(add_pp_letter_diff,        batched=True)
        ds_expanded = ds_expanded.map(add_contradiction_score,   batched=True)
        def add_sts_scores_eval(row):  return self._get_sts_scores_one_to_many(row['pp_l'], row['orig_sts_embeddings'])[0]
        df_sts['sts_scores'] = df_sts.apply(add_sts_scores_eval, axis=1)

        ## Merge together results 
        df_sts = df_sts.drop(columns = ['pp_l','orig_sts_embeddings'])
        df_sts_expanded = df_sts.apply(pd.Series.explode).reset_index(drop=True)
        ds_expanded = Dataset.from_pandas(ds_expanded.to_pandas().merge(df_sts_expanded, how='left', on='pp_idx').reset_index(drop=True))

        ## Calculate rewards and identify adversarial examples 
        def add_reward(batch): 
            batch['reward'] = self._get_reward(vm_scores=batch['vm_scores'], sts_scores=batch['sts_scores'],
                      pp_letter_diff=batch['pp_letter_diff'], contradiction_scores=batch['contradiction_scores']).cpu().tolist()
            return batch
        ds_expanded = ds_expanded.map(add_reward,   batched=True)
        def add_is_valid_pp(example): 
            example['is_valid_pp'] = self._is_valid_pp(sts_score=example['sts_scores'],
                 pp_letter_diff=example['pp_letter_diff'], contradiction_score=example['contradiction_scores'])*1
            return example 
        ds_expanded = ds_expanded.map(add_is_valid_pp,   batched=False)
        def add_is_adv_example(batch): 
            batch['is_adv_example'] = (np.array(batch['is_valid_pp']) * np.array(batch['label_flip'])).tolist()
            return batch
        ds_expanded = ds_expanded.map(add_is_adv_example,   batched=True)

        ## Calculate summary statistics
        df_expanded = ds_expanded.to_pandas()
        eval_metric_cols = ['label_flip', 'is_valid_pp', 'is_adv_example', 'reward', 'vm_scores', 'sts_scores',  'pp_letter_diff', 'contradiction_scores']
        agg_metrics = ['mean','std']  # not going to use the median 
        # avg across each orig 
        df_grp_stats = df_expanded[['idx'] + eval_metric_cols].groupby('idx').agg(agg_metrics)
        df_grp_stats.columns = df_grp_stats.columns = ["-".join(a) for a in df_grp_stats.columns.to_flat_index()]
        # avg across whole dataset 
        df_overall_stats = df_expanded[eval_metric_cols].groupby(lambda _ : True).agg(agg_metrics).reset_index(drop=True)
        df_overall_stats.columns = df_overall_stats.columns = ["-".join(a) + "-" + split for a in df_overall_stats.columns.to_flat_index()]
        df_overall_metrics = df_overall_stats.iloc[0].to_dict()   ## WANDB this 
        df_overall_metrics['any_adv_example_proportion' + "-" + split] = np.mean((df_grp_stats['is_adv_example-mean'] > 0 ) * 1)
        # add epoch key
        df_expanded['epoch'] = epoch
        df_overall_metrics['epoch'] = epoch

        ## Log results to wandb 
        if split in ['train', 'eval'] and not eval_ref_model: 
            wandb_eval_d = dict()
            mean_only = ['label_flip', 'is_valid_pp', 'is_adv_example']
            mean_and_std = ['reward', 'vm_scores', 'sts_scores', 'pp_letter_diff', 'contradiction_scores']
            for k in mean_only + mean_and_std: 
                name = k + "-mean"
                wandb_eval_d[name + "-"+ split + "-hist"] = Histogram(df_grp_stats[name].tolist())
            for k in mean_and_std:
                name = k + "-std"
                wandb_eval_d[name + "-" + split + "-hist"] = Histogram(df_grp_stats[name].tolist())
            wandb_eval_d = merge_dicts(df_overall_metrics, wandb_eval_d)
            wandb.log(wandb_eval_d, commit=True) 
        else: 
            pass
        
        # When to log to wandb? 
            # During each epoch of training loop for train and valid set 
            # Not for ref_model. Just save to CSV 
            # Not for test set. Just save to csv 

        ## Append paraphrase-level dataframe to file 
        fname = f"{self._cfg.path_run}{split}{'_ref_model' if eval_ref_model else ''}.csv"
        append_df_to_csv(df_expanded, path = fname)

            
    def _add_batch_vars_to_batch_d_eval(self,  raw, data, pp_l):
        self.batch_d = merge_dicts(self.batch_d, { 'idx': raw['idx'],
            'epoch': self.epoch, 'batch_num': self.batch_num, 'global_step': self.global_step,
            'acc_num': self.acc_num, "acc_batch_n_examples": self.acc_current_n_examples, 
            "orig_l": raw['text'], 
            "orig_label": data['label'].cpu().tolist(), 
            "orig_truelabel_probs": data['orig_truelabel_probs'].cpu().tolist(),
            'orig_length': self.orig_length, 'orig_batch_size': self.orig_batch_size, 
            "pp_l": pp_l, 'pp_length': self.pp_length, 'pp_batch_size': self.pp_batch_size
        })
        
    def _compute_and_log_eval_metrics(self): 
        """Calculate eval metrics for each split and log to wandb, then empty data_d"""
        wandb_d = dict(epoch=self.epoch)
        eval_splits = ["training_step", 'train', "valid"] if self.epoch != 0 else ['train', 'valid']
        for split in eval_splits: 
            # data d -> data frame 
            self.data_d[split] = self._convert_data_d_to_df(split)
            self._set_df_colorder(split)
            # calc metrics 
            df = self.data_d[split][['epoch'] + self._cfg.metrics]
            if split == "training_step": df = df.query("epoch == @self.epoch")
            d = df.mean()[self._cfg.metrics].to_dict()
            wandb_d = merge_dicts(wandb_d, {f"{k}_{split}": v for k, v in d.items()})
            # df append to file + empty data_d
            append_df_to_csv(self.data_d[split], path = f"{self._cfg.path_run}{split}.csv")
            self.data_d[split] = [] 
        wandb.log(wandb_d, commit=True)    
    
    def _set_df_colorder(self, data_d_key): 
        colorder_eval=['idx','epoch', 'orig_l',  'pp_l','orig_truelabel_probs','pp_truelabel_probs',
        'pp_predclass_probs','orig_label','pp_predclass','label_flip', 'vm_score','sts_score', 'pp_letter_diff', 'pp_letter_percent',  
        'contradiction_score', 'reward', 'pp_logp','ref_logp', 'kl_div', 'reward_penalty',  'reward_with_penalty','loss','batch_num',
                       'global_step','acc_num','loss_sum', 'loss_batch', 'label_flip_fraction',
                       'orig_length','orig_batch_size','pp_length','pp_batch_size']
        if data_d_key == "training_step": 
            colorder_training_step = colorder_eval + [o for o in self.data_d['training_step'].columns if 'time_' in o]
            assert len(set(colorder_training_step).difference(set(self.data_d[data_d_key].columns))) == 0 
            self.data_d[data_d_key] = self.data_d[data_d_key][colorder_training_step]
        else:
            assert len(set(colorder_eval).difference(set(self.data_d[data_d_key].columns))) == 0 
            self.data_d[data_d_key] = self.data_d[data_d_key][colorder_eval]

    def _add_wandb_run_summary_statistics(self):
        """Compute test metrics for the run and log them to the wandb run summary pane. """
        ## Summary statistics of the test set 
        # From the last epoch atm because we don't have early stopping 
        test_metrics = self.data_d['test'].filter(self._cfg.metrics, axis=1).mean()
        for metric, val in zip(test_metrics.index, test_metrics): 
            self.run.summary[f"{metric}_avg_test"] = val        
    
    def _get_gradient_update_norm(self):
        total_norm = 0
        for p in [o for o in self.pp_model.parameters() if o[1].requires_grad]:
            if p.grad is not None:  # the embed_position layers on encoder/decoder dont keep grad ()
                param_norm = p.grad.detach().data.norm(2)
                total_norm += param_norm.item() ** 2
        total_norm = total_norm ** 0.5
        return total_norm
    
    def _plot_grad_flow(self, named_parameters):
        '''Plots the gradients flowing through different layers in the net during training.
        Can be used for checking for possible gradient vanishing / exploding problems.'''
        plt.figure()
        ave_grads = []
        max_grads= []
        layers = []
        for n, p in named_parameters:
            if(p.requires_grad) and ("bias" not in n):
                layers.append(n)
                ave_grads.append(p.grad.abs().mean())
                max_grads.append(p.grad.abs().max())
        plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.1, lw=1, color="c")
        plt.bar(np.arange(len(max_grads)), ave_grads, alpha=0.1, lw=1, color="b")
        plt.hlines(0, 0, len(ave_grads)+1, lw=2, color="k" )
        plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
        plt.xlim(left=0, right=len(ave_grads))
        plt.ylim(bottom = -0.001, top=0.02) # zoom in on the lower gradient regions
        plt.xlabel("Layers")
        plt.ylabel("Average Gradient")
        plt.title("Gradient Flow")
        plt.grid(True)
        plt.legend([Line2D([0], [0], color="c", lw=4),
                    Line2D([0], [0], color="b", lw=4),
                    Line2D([0], [0], color="k", lw=4)], ['max-gradient', 'mean-gradient', 'zero-gradient'])
        return plt

In [ ]:
f"{'_ref_model' if True else ''}.csv"

'_ref_model.csv'

In [ ]:
f'{"hello"if True else 2}'


'hello'

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 02_tests.ipynb.
Converted 03_config.ipynb.
Converted 07_models.ipynb.
Converted 10_data.ipynb.
Converted 20_trainer.ipynb.
Converted 25_insights.ipynb.
Converted baselines.ipynb.
Converted baselines_analysis.ipynb.
Converted index.ipynb.
Converted pp_eval_baselines.ipynb.
Converted run.ipynb.
Converted show_examples.ipynb.
Converted test_pp_model.ipynb.
